In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:

#載入數據集，one_hot是把数据转化为只有0和1的形式
mnist = input_data.read_data_sets("MNIST_data", one_hot = True) #這步有時候會失效

#因為數據集很大，故我們要用stochastic gradient descent，
#會將資料集分批次（一次100张）放入神经网络进行训练，
#並不會一次將所有資料拿來train (計算量很大)
#每一個批次的大小
batch_size = 100 

#計算一共有多少批次，训练集数量mnist.train.num_examples 
# // 在python中表示取商
n_batch = mnist.train.num_examples // batch_size      

#定义兩個placeholder，目的在於 train時候透過 feed 傳入 x_data 與 y_data
x = tf.placeholder(tf.float32, [None, 784]) # 28 * 28 = 784，None值变为100
y = tf.placeholder(tf.float32, [None, 10]) #輸出層，有十個神經元，每個神經元有一個激活值，十個激活值排成一個 1*10的向量
keep_prob = tf.placeholder(tf.float32)

#创建一個簡單的神經網路 (只有输入层和輸出層，输入层784个神经元，输出层總共10個神經元，即十个标签)
# 新的初始化方法，stddev标准差
#隐藏层1（2000个神经元）
W1 = tf.Variable(tf.truncated_normal([784, 2000], stddev=0.1))
b1 = tf.Variable(tf.zeros([2000])+0.1)
L1 = tf.nn.tanh(tf.matmul(x, W1) + b1)
L1_drop = tf.nn.dropout(L1, keep_prob)    #tensorflow封装好的dropout函数

#隐藏层2（2000个神经元）
W2 = tf.Variable(tf.truncated_normal([2000, 2000], stddev=0.1))
b2 = tf.Variable(tf.zeros([2000])+0.1)
L2 = tf.nn.tanh(tf.matmul(L1_drop, W2) + b2)
L2_drop = tf.nn.dropout(L2, keep_prob)

#隐藏层3 （1000个神经元）
W3 = tf.Variable(tf.truncated_normal([2000, 1000], stddev=0.1))
b3 = tf.Variable(tf.zeros([1000])+0.1)
L3 = tf.nn.tanh(tf.matmul(L2_drop, W3) + b3)
L3_drop = tf.nn.dropout(L3, keep_prob)


#输出层4 （10个神经元）
#（实际上并不需要定义这么多神经元，此处只是为了模拟过拟合现象）
#过拟合现象出现的原因有二：一是网络太复杂，二是数据量太小
#网络模型有很多的参数，不好确定，未知数太多，已知的公式太少，求不出应有的解
W4 = tf.Variable(tf.truncated_normal([1000, 10], stddev=0.1))
b4 = tf.Variable(tf.zeros([10])+0.1)
#L4 = tf.nn.tanh(tf.matmul(L3_drop, W4) + b4)
#L4_dropout = tf.nn.dropout(L4, keep_prob)
prediction = tf.nn.softmax(tf.matmul(L3_drop, W4) + b4) 

#W1 = tf.Variable(tf.zeros([784, 10]))              #权值
    # b = tf.Variable(tf.zeros([1, 10]))                #偏置值
#b1 = tf.Variable(tf.zeros([10]))  
#prediction = tf.nn.softmax(tf.matmul(x, W) + b)   #预测值，用到softmax


#二次代價函數 : loss = mean((y - prediction)^2)
#loss = tf.reduce_mean(tf.square(y - prediction))
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y, logits = prediction)) #交叉熵

#使用梯度下降法
#Gradient desent method  (learning rate = 0.2)
gd = tf.train.GradientDescentOptimizer(0.2)

#最小化 代價函數 (operator)
train_step = gd.minimize(loss)
#以上两句可以合并为 train_step = tf.train.GradientDescentOptimizer(0.2).minimize(loss)

#初始化变量 operator
init = tf.global_variables_initializer()



#测试训练的准确率，求准确率的方法
#如果y標籤最大的值，與prediction標籤最大的值相等，則回傳true
#結果存在一個 boolean 的变量correct_prediction中

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))
                              
 #argmax 返回一維張量中最大的值所在的位置
 # 求标签y里面最大的值在哪个位置即标签
 # tf.argmax(prediction, 1)预测 概率最大就会判定识别的这张图片是属于哪个标签的
 # (tf.argmax(y, 1)，真实样本的y存放的都是0或1，哪位是1就会返回哪位的值
 #  然后再比较上面两者，是否一样
                             
                              
# 求准确率
# 轉換資料格式 boolean 轉成 32位的float，接著再取平均值，得到准确率
# true转换为1.0，false转换为0
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))         
                              
#開始training
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(31): #总共疊代21次（周期） (outer loop)，把所有的图片训练21次
        #每一次 outer loop 不一次拿所有的數據集，來做 Gradient desent，這就是 stochastic gradient descent
        for batch in range(n_batch):#每一個 outer loop 疊代 n_batch 個批次
            #利用 train.next_batch 函數，讀取一個batch的 x, y 存給 batch_xs图片数据, batch_ys图片标签
            # mnist.train.next_batch(batch_size)是获取下一个一百张图片               
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)  
              
           # keep_prob:1.0意思是所有的神经元都工作，此时dropout是没有用的    
            feed_dict = {x: batch_xs, y: batch_ys, keep_prob:0.7}  #拿來feed 的 dictionary                  
            sess.run(train_step, feed_dict)      # keep_prob:0.7是70%的神经元是工作的
                              
        #每做完一次 outer loop 計算一次准确率
        #测试数据计算出的准确率
        outer_loop_feed_dict = {x: mnist.test.images, y: mnist.test.labels, keep_prob:1.0} #testing data feed dictionary
        test_acc = sess.run(accuracy, outer_loop_feed_dict)
          
         #训练集做测试算出的准确率   
        outer_loop_feed_dict1 = {x: mnist.train.images, y: mnist.train.labels, keep_prob:1.0} #testing data feed dictionary
        train_acc = sess.run(accuracy, outer_loop_feed_dict1)
        #测试图片训练，测试图片测试，得到的一般是好的准确率
        
        # str(epoch)周期数
        print("Iter =" + str(epoch) + ", Testing Accuracy =" + str(test_acc)+",Training Accuracy=" + str(train_acc))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Iter =0, Testing Accuracy =0.9184,Training Accuracy=0.9136909
Iter =1, Testing Accuracy =0.9287,Training Accuracy=0.9259091
Iter =2, Testing Accuracy =0.9388,Training Accuracy=0.9361454
Iter =3, Testing Accuracy =0.9406,Training Accuracy=0.94223636
Iter =4, Testing Accuracy =0.9453,Training Accuracy=0.9461091
Iter =5, Testing Accuracy =0.9474,Training Accuracy=0.9502182
Iter =6, Testing Accuracy =0.949,Training Accuracy=0.95245457
Iter =7, Testing Accuracy =0.9501,Training Accuracy=0.9554
Iter =8, Testing Accuracy =0.9526,Training Accuracy=0.95754546
Iter =9, Testing Accuracy =0.9548,Training Accuracy=0.9598182
Iter =10, Testing Accuracy =0.9561,Training Accuracy=0.9605455
Iter =11, Testing Accuracy =0.9571,Training Accuracy=0.96230906
Iter =12, Testing Accuracy =0.9602,Training Accuracy=0.96469

In [ ]:
#keep_prob:0.7之后，收敛速度变慢了，30次重复之后还没有达到最优的结果，还有提升空间
# 使用drop_out之后，Testing Accuracy 和 Training Accuracy的差距变得很小
# 没有使用的时候，Testing Accuracy 和 Training Accuracy偏差很大的
Iter =0, Testing Accuracy =0.9184,Training Accuracy=0.9136909
Iter =1, Testing Accuracy =0.9287,Training Accuracy=0.9259091
Iter =2, Testing Accuracy =0.9388,Training Accuracy=0.9361454
Iter =3, Testing Accuracy =0.9406,Training Accuracy=0.94223636
Iter =4, Testing Accuracy =0.9453,Training Accuracy=0.9461091
Iter =5, Testing Accuracy =0.9474,Training Accuracy=0.9502182
Iter =6, Testing Accuracy =0.949,Training Accuracy=0.95245457
Iter =7, Testing Accuracy =0.9501,Training Accuracy=0.9554
Iter =8, Testing Accuracy =0.9526,Training Accuracy=0.95754546
Iter =9, Testing Accuracy =0.9548,Training Accuracy=0.9598182
Iter =10, Testing Accuracy =0.9561,Training Accuracy=0.9605455
Iter =11, Testing Accuracy =0.9571,Training Accuracy=0.96230906
Iter =12, Testing Accuracy =0.9602,Training Accuracy=0.9646909
Iter =13, Testing Accuracy =0.9592,Training Accuracy=0.96476364
Iter =14, Testing Accuracy =0.9615,Training Accuracy=0.96605456
Iter =15, Testing Accuracy =0.9615,Training Accuracy=0.9668546
Iter =16, Testing Accuracy =0.9615,Training Accuracy=0.9676545
Iter =17, Testing Accuracy =0.9641,Training Accuracy=0.9691091
Iter =18, Testing Accuracy =0.965,Training Accuracy=0.9698909
Iter =19, Testing Accuracy =0.9661,Training Accuracy=0.97083634
Iter =20, Testing Accuracy =0.9674,Training Accuracy=0.97196364
Iter =21, Testing Accuracy =0.9664,Training Accuracy=0.9727455
Iter =22, Testing Accuracy =0.9665,Training Accuracy=0.9724727
Iter =23, Testing Accuracy =0.967,Training Accuracy=0.9735091
Iter =24, Testing Accuracy =0.9688,Training Accuracy=0.9742182
Iter =25, Testing Accuracy =0.9679,Training Accuracy=0.9748909
Iter =26, Testing Accuracy =0.9695,Training Accuracy=0.9755273
Iter =27, Testing Accuracy =0.9689,Training Accuracy=0.97636366
Iter =28, Testing Accuracy =0.9701,Training Accuracy=0.97638184
Iter =29, Testing Accuracy =0.9693,Training Accuracy=0.9778
Iter =30, Testing Accuracy =0.9702,Training Accuracy=0.97825456